# Shortest Path between two entries of Wikipedia's Network

#### Imports

In [2]:
import requests # That's to get the links from a wikipedia page
import os, time
import numpy as np
import networkx as nx # Networkx is more convenient to analyze graphs

### 1. Get all child links from a page

In [3]:
def all_links(title, language, plcontinue=None):
    url = "https://{lang}.wikipedia.org/w/api.php".format(lang=language)
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "links",
        "pllimit": "max",
        "plnamespace": 0,
    }
    if plcontinue:
        params["plcontinue"] = plcontinue

    data = requests.get(url, params=params)
    data = data.json()
    plcontinue = data.get("continue", {}).get("plcontinue", None)
    for items in data["query"]["pages"].values():
        for link in items.get("links", []):
            yield link["title"]

    if plcontinue is not None:
        yield from all_links(title, language, plcontinue=plcontinue)

A function to clean the children links from a page from "rubbish", but this isn't used because of the
difficulty of defining "rubbish".

In [4]:
def clean_entries(rawlist):
    ''' This function takes a generator as input ! 
        Clears the entries starting with a '.', as these are language links.
    '''
    rawlist = np.array(list(rawlist))
    L = rawlist.size
    
    if L == 0:
        return rawlist
    else:
        lowcut = 0
        while (rawlist[lowcut][0] == "."):
            lowcut += 1
            if lowcut == L:
                return np.array([])
                
        return rawlist[lowcut:]

### 2. Explore the network(s) by doing a BFS

In [5]:
def explore_network(language, initial_entry, target_entry, restrictions_dic):
    ''' Inputs :
            - language -> [string]; either "en", "fr", or "de", language we browse wikipedia in,
            - initial_entry -> [string]; Wikipedia page we start the exploration from,
            - target_entry -> [string]; Wikipedia page we want to reach,
            - restrictions -> [dictionary]; restrictions to avoid browsing the whole network
        Outputs :
            - A networkx newtork object where the nodes are Wikipedia pages and the edges the links between them.
        '''
    
    # Strings used for interactive output
    outstr1 = r"*–> Took {} (total {}) seconds to process 500 more nodes (total {}) !"
    outstr2 = r"*–> Currently at node {}, layer {}, still {} to go !"
    
    # Recover the restrictions
    maxdepth = restrictions_dic["max_depth"]
    maxnodes = restrictions_dic["max_nodes"]
    
    # Initialize the BFS
    u = initial_entry # node we start the exploration from
    queue = [u] # the queue of nodes to be processed
    n = 0 # order in which the nodes are added to the queue and thus the graph
    nprocessed = 0 # nb of processed nodes
    
    # Initialize the networkx network
    G = nx.DiGraph()
    G.add_node(u)
    G.nodes[u]["order"] = n
    G.nodes[u]["layer"] = 0
    G.nodes[u]["parent"] = u
    
    # Run a BFS for as many layers as needed
    t0 = time.time()
    t1 = time.time()
    while (queue != []):
        # Select a new source node
        u = queue[0]
        u_neighbors = all_links(u, language) # note that this is a generator !
        
        # A little message to wait before the graph finishes completing once we've seen enough nodes
        if (nprocessed == maxnodes) and (G.nodes[u]["order"] % 100 == 0):           
                        print(r"|")
                        print(outstr2.format(G.nodes[u]["order"],  G.nodes[u]["layer"], len(queue)))  
            
        # Explore the neighbors of the new source.
        # Note that if the generator is empty then nothing happens and "queue" is poped -> good !
        for v in u_neighbors:
            # Check if the node has already been seen (and if the node is not a language link)
            if (v not in G.nodes()) and (v[0] != "."):
                # End the process if we reached the desired node
                if v == target_entry:
                    print("Finished !")
                    return G
                # Check if we've attained the maximum amount of explorable nodes or the maximum allowed depth
                if (nprocessed < maxnodes) and (G.nodes[u]["layer"] < maxdepth):
                    G.add_edge(u, v)
                    G.nodes[v]["order"] = n + 1
                    G.nodes[v]["layer"] = G.nodes[u]["layer"] + 1
                    G.nodes[v]["parent"] = u
                    queue.append(v)
                    n += 1
                    nprocessed += 1

                    # A little message every time we process more than 500 nodes
                    if (nprocessed % 10000 == 0) and (nprocessed > 0):           
                        t2 = time.time()
                        deltat = t2 - t1
                        print(r"|")
                        print(outstr1.format(deltat, (t2 - t0), nprocessed))
                        print(outstr2.format(G.nodes[u]["order"],  G.nodes[u]["layer"], len(queue)))
                        t1 = time.time()
                              
        # Dequeue the node once all its neighbors have been seen
        queue.pop(0)
    
    t3 = time.time()
    deltat = t3 - t0
    print("Took {} seconds to build the whole network !".format(deltat))

    return G

In [6]:
restrictions = {"max_depth" : 10, # How deep in the "BFS" we are allowed to go
                "max_nodes" : int(2e6), # Maximum of nodes EXPLORED (and not necessarily part of the network)
               }

In [7]:
# Define the two nodes whose distance we want to find
base_entry = "France"
target_entry = "Germany"
#TODO -> It might be useful to define a function that checks if those entries are true Wikipedia pages 
#    (in particular to avoid spelling mistakes making a page impossible to reach.)

# Define a path to which the graph will be written
cwd = os.getcwd()
basepath = "{cwd}/network-{baseentry}-{targetentry}_{maxnodes}-{maxlayer}.{ext}"

# Explore the Network
language = "en"
print()
print("Finding shortest path from base entry \"{}\" to target entry \"{}\"".format(base_entry, target_entry))
N = explore_network(language, base_entry, target_entry, restrictions)

# Save the network to different file formats
print("Saving network ...")
filename = basepath.format(cwd=cwd, baseentry=base_entry, targetentry=target_entry,
                           maxnodes=restrictions["max_nodes"],
                           maxlayer=restrictions["max_depth"], 
                           ext="gml")
nx.write_gml(N, filename)


Finding shortest path from base entry "France" to target entry "Germany"
Finished !
Saving network ...
